In [176]:
"""
This file performs preprocessing on the relevant data, to prepare for model building.
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import numpy as np
from typing import Union, List
import xgboost

from sklearn import preprocessing, tree, ensemble, linear_model, metrics, model_selection, svm


In [177]:
def twentyfour_hour_to_float(HHMM: str) -> float:
    """
    Converts a time in HHMM format to a float representing hours.
    
    Parameters
    ----------
    HHMM : str
        A string representing time in HHMM format (e.g., '1230' for 12:30 PM).
    
    Returns:
    ----------
    float
        A float representing the time in hours. If HHMM is '-', returns NaN.
    """

    if HHMM == '-':
        return np.nan
    else:
        hour = float(HHMM[:2])
        minute = float(HHMM[2:])
        return hour + minute/60
    

def avg_wind_direction(S1: Union[float, np.nan], S2: Union[float, np.nan]) -> float:
    """
    Calculates the average wind direction from two stations, accounting for circular nature of wind direction.

    Parameters
    ----------
    S1 : float or np.nan
        Wind direction from Station 1 (in 10-degree increments, 0-35 scale).
    S2 : float or np.nan
        Wind direction from Station 2 (in 10-degree increments, 0-35 scale).

    Returns
    -------
    float
        Averaged wind direction. Handles missing values and circular differences.
    """

    # Handle missing values
    if S1 == np.nan:
        avg = float(S2)
    elif S2 == np.nan:
        avg = float(S1)
        
    # Handle circular difference (i.e., crossing the 0/36 boundary)
    elif np.abs(S1 - S2) > 18:
        if S1 < S2:
            S1 += 36
        else:
            S2 += 36
        avg = (S1 + S2) / 2.0
    else:
        avg = (S1 + S2) / 2.0

    # Adjust back if average exceeds 36 (due to circular adjustment)
    if avg >= 36:
        avg -= 36
        
    return avg


def combine_conditions(S1: str, S2: str) -> Union[str, List[str]]:
    """
    Combines weather condition codes from two stations.

    Parameters
    ----------
    S1 : str
        Condition string from Station 1 (space-separated codes).
    S2 : str
        Condition string from Station 2 (space-separated codes).

    Returns
    -------
    str or list
        Combined unique condition codes. Returns empty string if both are empty.
    """

    # Handle empty conditions from both stations
    if S1 == " " and S2 == " ":
        return ""
    elif S1 == " ":
        return S2
    elif S2 == " ":
        return S1
    else:
        # Split condition codes into lists and combine unique codes
        S1_list = S1.split(" ")
        S2_list = S2.split(" ")
        return list(set(S1_list + S2_list))


def avg_col(S1: Union[float, np.nan], S2: Union[float, np.nan]) -> float:
    """
    Computes the average of two numeric values, handling missing data.

    Parameters
    ----------
    S1 : float or np.nan
        Value from Station 1.
    S2 : float or np.nan
        Value from Station 2.

    Returns
    -------
    float
        Average of S1 and S2, or the non-missing value if one is missing.
    """

    # Handle missing values
    if S1 == np.nan:
        avg = float(S2)
    elif S2 == np.nan:
        avg = float(S1)
    else:
        avg = (S1 + S2) / 2.0
        
    return avg


def find_zip(address: str) -> str:
    """
    Extracts the ZIP code from an address string.

    Parameters
    ----------
    address : str
        The address string containing the ZIP code.

    Returns
    -------
    str
        The extracted ZIP code if found, otherwise an empty string.
    """
    
    if re.search('(?<=IL )[0-9]*', address):
        return re.search('(?<=IL )[0-9]*', address).group(0)

In [178]:
# TODO: Incorporate spray data
spray = pd.read_pickle("../data/spray.pkl")
weather = pd.read_pickle("../data/weather.pkl")
mosquito = pd.read_pickle("../data/mosquito_data.pkl")

In [179]:
# Convert dates in each file to datetime objects
for df in [spray, weather, mosquito]:
    df.Date = pd.to_datetime(df.Date, format='%Y-%m-%d')

In [180]:
# In a similar way, many other variables in the weather dataframe are objects due to the values that
# some observations have, such as "M" for missing values. These will be converted to numeric.
num_vars = ['Tavg', 'Depart', 'DewPoint', 'WetBulb', 'Heat', 'Cool', 'PrecipTotal',
                 'StnPressure', 'SeaLevel', 'ResultSpeed', 'ResultDir', 'AvgSpeed']
for var in num_vars:
    weather[var] = pd.to_numeric(weather[var], errors='coerce')

In [181]:
# Extracting time features for time columns
weather.Sunrise = weather.Sunrise.apply(twentyfour_hour_to_float)
weather.Sunset = weather.Sunset.apply(twentyfour_hour_to_float)

# Calculate hours in day
weather["hours_in_day"] = weather.Sunset - weather.Sunrise

In [182]:
# Dropping unnecessary columns since they are mostly all NaNs
weather.drop(['Depth', 'Water1', 'SnowFall'], axis=1, inplace=True)

In [183]:
# For simplicity, the data from both weather stations will be combined

# Initialize dataframe to hold combined weather data
combined_weather = pd.DataFrame()

# Split the weather data into two stations
s1 = weather[weather.Station==1]
s2 = weather[weather.Station==2]

# Set columns that only have data for Station 1 or the data being the same.
for col in ['Date', 'Depart', 'Sunrise', 'Sunset', 'hours_in_day']:
    combined_weather[col] = s1[col].values

# Apply average functions to the columns that have data for both stations
for col in ['Tmax', 'Tmin', 'Tavg', 'DewPoint', 'WetBulb', 'Heat', 'Cool', 'StnPressure', 'SeaLevel', 'ResultSpeed', 'AvgSpeed']:
    combined_weather[col] = [avg_col(a, b) for a, b in zip(s1[col].values, s2[col].values)]

# Applying related weather average functions
combined_weather['ResultDir'] = [avg_wind_direction(a, b) for a, b in zip(s1.ResultDir.values, s2.ResultDir.values)]
combined_weather['CodeSum'] = [combine_conditions(a, b) for a, b in zip(s1.CodeSum.values, s2.CodeSum.values)]

# PrecipTotal for each station is too different, so I keep both separate.
combined_weather['PrecipTotal_station1'] = weather.PrecipTotal[weather.Station==1].values
combined_weather['PrecipTotal_station2'] = weather.PrecipTotal[weather.Station==2].values

In [184]:
# However, the precipitation totals have too many missing values to be worth imputing in an accurate 
# sense. They will be dropped here.
combined_weather.drop(['PrecipTotal_station1', 'PrecipTotal_station2'], axis=1, inplace=True)

In [186]:
# Here, I impute missing values for some columns. I try using a unique strategy, where I train a 
# small model to predict the missing values based on the other columns. This is done for the
# WetBulb and StnPressure columns, which have a significant number of missing values.
"""
# WetBulb
X = combined_weather.dropna().drop(['Date', 'WetBulb', 'CodeSum'], axis=1)
y = combined_weather.dropna()['WetBulb']

dt_WetBulb = ensemble.RandomForestRegressor(n_estimators=100)
dt_WetBulb.fit(X, y)

# Predict missing WetBulb values
mask_wb_null = combined_weather['WetBulb'].isnull()
X_pred = combined_weather.loc[mask_wb_null, X.columns]
predicted_WetBulb = dt_WetBulb.predict(X_pred)

combined_weather.loc[mask_wb_null, 'WetBulb'] = predicted_WetBulb

# StnPressure
X = combined_weather.dropna().drop(['Date', 'StnPressure', 'CodeSum'], axis=1)
y = combined_weather.dropna()['StnPressure']

xb_StnPressure = xgboost.XGBRegressor()
xb_StnPressure.fit(X, y)

# Predict missing StnPressure values
mask_sp_null = combined_weather['StnPressure'].isnull()
X_pred = combined_weather.loc[mask_sp_null, X.columns]
predicted_StnPressure = xb_StnPressure.predict(X_pred)

combined_weather.loc[mask_sp_null, 'StnPressure'] = predicted_StnPressure
"""

"\n# WetBulb\nX = combined_weather.dropna().drop(['Date', 'WetBulb', 'CodeSum'], axis=1)\ny = combined_weather.dropna()['WetBulb']\n\ndt_WetBulb = ensemble.RandomForestRegressor(n_estimators=100)\ndt_WetBulb.fit(X, y)\n\n# Predict missing WetBulb values\nmask_wb_null = combined_weather['WetBulb'].isnull()\nX_pred = combined_weather.loc[mask_wb_null, X.columns]\npredicted_WetBulb = dt_WetBulb.predict(X_pred)\n\ncombined_weather.loc[mask_wb_null, 'WetBulb'] = predicted_WetBulb\n\n# StnPressure\nX = combined_weather.dropna().drop(['Date', 'StnPressure', 'CodeSum'], axis=1)\ny = combined_weather.dropna()['StnPressure']\n\nxb_StnPressure = xgboost.XGBRegressor()\nxb_StnPressure.fit(X, y)\n\n# Predict missing StnPressure values\nmask_sp_null = combined_weather['StnPressure'].isnull()\nX_pred = combined_weather.loc[mask_sp_null, X.columns]\npredicted_StnPressure = xb_StnPressure.predict(X_pred)\n\ncombined_weather.loc[mask_sp_null, 'StnPressure'] = predicted_StnPressure\n"

In [187]:
mosquito.Date

0        2007-05-29
1        2007-05-29
2        2007-05-29
3        2007-05-29
4        2007-05-29
            ...    
116288   2014-10-02
116289   2014-10-02
116290   2014-10-02
116291   2014-10-02
116292   2014-10-02
Name: Date, Length: 126799, dtype: datetime64[ns]

Train/valid/test split

In [188]:
# Since the mosquito data is time based, it would be bad to split randomly, since this can cause 
# data leakage. Instead, I will split the data into train, validation, and test sets based on time.
# Train data covers 2007-2011, validation data covers 2012, and test data covers 2013 onwards.
# (This is roughly a 62.5%/12%/25% split.)
train = mosquito[mosquito['Date'] < '2012-01-01']
valid = mosquito[(mosquito['Date'] >= '2012-01-01') & (df['Date'] < '2013-01-01')]
test  = mosquito[mosquito['Date'] >= '2013-01-01']

In [189]:
# Extract zip codes; this will be the main location based feature outside of lat/long
train['zip_code'] = train.Address.apply(find_zip)
valid['zip_code'] = valid.Address.apply(find_zip)
test['zip_code'] = test.Address.apply(find_zip)

# Address, Street, and AddressNumberAndStreet provide the same information, so they are removed to 
# avoid redundancy.
train = train.drop(['Address', 'Street', 'AddressNumberAndStreet'], axis=1)
valid = valid.drop(['Address', 'Street', 'AddressNumberAndStreet'], axis=1)
test = test.drop(['Address', 'Street', 'AddressNumberAndStreet'], axis=1)

C:\Users\kevin\AppData\Local\Temp\ipykernel_9368\1888388000.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['zip_code'] = train.Address.apply(find_zip)
C:\Users\kevin\AppData\Local\Temp\ipykernel_9368\1888388000.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid['zip_code'] = valid.Address.apply(find_zip)
C:\Users\kevin\AppData\Local\Temp\ipykernel_9368\1888388000.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

In [190]:
# Imputing missing zip codes based on lat/long. A unique strategy is used here, where I train a 
# model to do this based on the training latitude and longitude data.
X = train.dropna(subset=['Latitude', 'Longitude', 'zip_code'])[['Latitude', 'Longitude']]
y = train.dropna(subset=['Latitude', 'Longitude', 'zip_code'])['zip_code']

# Fit model to predict zip codes based on lat/long
dtree_forzip = tree.DecisionTreeClassifier()
dtree_forzip.fit(X, y)

# Define a mask to look for rows with missing zip codes, and predict them for train set
mask_zip_null = train['zip_code'].isnull()
X_missing_zip = train.loc[mask_zip_null, ['Latitude', 'Longitude']]
predicted_zip = dtree_forzip.predict(X_missing_zip)
train.loc[mask_zip_null, 'zip_code'] = predicted_zip

# Repeat for valid and test sets
mask_zip_null = valid['zip_code'].isnull()
X_missing_zip = valid.loc[mask_zip_null, ['Latitude', 'Longitude']]
predicted_zip = dtree_forzip.predict(X_missing_zip)
valid.loc[mask_zip_null, 'zip_code'] = predicted_zip

mask_zip_null = test['zip_code'].isnull()
X_missing_zip = test.loc[mask_zip_null, ['Latitude', 'Longitude']]
predicted_zip = dtree_forzip.predict(X_missing_zip)
test.loc[mask_zip_null, 'zip_code'] = predicted_zip

In [191]:
# Extract day of year, month, day of week, and year from date column
train['day_of_year'] = train.Date.dt.dayofyear
train['month'] = train.Date.dt.month
train['day_of week'] = train.Date.dt.dayofweek
train['year'] = train.Date.dt.year

valid['day_of_year'] = valid.Date.dt.dayofyear
valid['month'] = valid.Date.dt.month    
valid['day_of week'] = valid.Date.dt.dayofweek
valid['year'] = valid.Date.dt.year

test['day_of_year'] = test.Date.dt.dayofyear
test['month'] = test.Date.dt.month
test['day_of week'] = test.Date.dt.dayofweek
test['year'] = test.Date.dt.year

In [192]:
# Here, I create some new features based on trap data. The idea is to create features that capture 
# the distribution of mosquitos and WNV presence across different traps. This can help the model 
# understand the relative importance of different traps in terms of mosquito counts and WNV 
# presence.
num_by_trap = train[['Trap', 'NumMosquitos', 'WnvPresent']].groupby('Trap').agg('sum')
num_by_trap['trap_percent_of_all_mosquitos'] = num_by_trap['NumMosquitos']/sum(num_by_trap.NumMosquitos)
num_by_trap['trap_percent_with_wnv'] = num_by_trap.WnvPresent/num_by_trap.NumMosquitos
num_by_trap.reset_index(inplace=True)

# Define dictionaries to map trap names to their respective weights
map_mosq_weight = {t:v for t, v in zip(num_by_trap.Trap.values, num_by_trap['trap_percent_of_all_mosquitos'].values)}
map_wnv_weight = {t:v for t, v in zip(num_by_trap.Trap.values, num_by_trap['trap_percent_with_wnv'].values)}

# Create a second mapping for WNV presence, which is the average per-record WNV presence per trap
map_wnv_weight2 = {}
for trap in set(train.Trap):
    map_wnv_weight2[trap] = sum(train.WnvPresent[train.Trap==trap]/sum(train.Trap==trap))
map_wnv_weight2

# Map the trap weights to the train, valid, and test sets
train['trap_percent_of_all_mosquitos'] = train.Trap.map(map_mosq_weight)
train['trap_percent_with_wnv'] = train.Trap.map(map_wnv_weight)
train['trap_percent_with_wnv2'] = train.Trap.map(map_wnv_weight2)

valid['trap_percent_of_all_mosquitos'] = valid.Trap.map(map_mosq_weight).fillna(0)
valid['trap_percent_with_wnv'] = valid.Trap.map(map_wnv_weight).fillna(0)
valid['trap_percent_with_wnv2'] = valid.Trap.map(map_wnv_weight2).fillna(0)

test['trap_percent_of_all_mosquitos'] = test.Trap.map(map_mosq_weight).fillna(0)
test['trap_percent_with_wnv'] = test.Trap.map(map_wnv_weight).fillna(0)
test['trap_percent_with_wnv2'] = test.Trap.map(map_wnv_weight2).fillna(0)

In [193]:
# Join the mosquito data with the combined weather data, left joining on the date column
train_w = pd.merge(train, combined_weather, how='left', on='Date')
valid_w = pd.merge(valid, combined_weather, how='left', on='Date')
test_w = pd.merge(test, combined_weather, how='left', on='Date')

In [194]:
# Impute missing values for WetBulb and StnPressure using a model-based approach, similar to above

# WetBulb
X = combined_weather.dropna().drop(['Date', 'WetBulb', 'CodeSum'], axis=1)
y = combined_weather.dropna()['WetBulb']

dt_WetBulb = ensemble.RandomForestRegressor(n_estimators=100)
dt_WetBulb.fit(X, y)

# Predict missing WetBulb values
mask_wb_null = train_w['WetBulb'].isnull()
X_pred = train_w.loc[mask_wb_null, X.columns]
predicted_WetBulb = dt_WetBulb.predict(X_pred)
train_w.loc[mask_wb_null, 'WetBulb'] = predicted_WetBulb

mask_wb_null = valid_w['WetBulb'].isnull()
X_pred = valid_w.loc[mask_wb_null, X.columns]
predicted_WetBulb = dt_WetBulb.predict(X_pred)
valid_w.loc[mask_wb_null, 'WetBulb'] = predicted_WetBulb

mask_wb_null = test_w['WetBulb'].isnull()
X_pred = test_w.loc[mask_wb_null, X.columns]
predicted_WetBulb = dt_WetBulb.predict(X_pred)
test_w.loc[mask_wb_null, 'WetBulb'] = predicted_WetBulb

# StnPressure
X = combined_weather.dropna().drop(['Date', 'StnPressure', 'CodeSum'], axis=1)
y = combined_weather.dropna()['StnPressure']

xb_StnPressure = xgboost.XGBRegressor()
xb_StnPressure.fit(X, y)

# Predict missing StnPressure values
mask_sp_null = train_w['StnPressure'].isnull()
X_pred = train_w.loc[mask_sp_null, X.columns]
predicted_StnPressure = xb_StnPressure.predict(X_pred)
train_w.loc[mask_sp_null, 'StnPressure'] = predicted_StnPressure

mask_sp_null = valid_w['StnPressure'].isnull()
X_pred = valid_w.loc[mask_sp_null, X.columns]
predicted_StnPressure = xb_StnPressure.predict(X_pred)
valid_w.loc[mask_sp_null, 'StnPressure'] = predicted_StnPressure

mask_sp_null = test_w['StnPressure'].isnull()
X_pred = test_w.loc[mask_sp_null, X.columns]
predicted_StnPressure = xb_StnPressure.predict(X_pred)
test_w.loc[mask_sp_null, 'StnPressure'] = predicted_StnPressure

ValueError: Found array with 0 sample(s) (shape=(0, 15)) while a minimum of 1 is required by RandomForestRegressor.

In [196]:
print("Missing values in train_w:")
print(train_w.isnull().sum()[train_w.isnull().sum() > 0])

print("\nMissing values in valid_w:")
print(valid_w.isnull().sum()[valid_w.isnull().sum() > 0])

print("\nMissing values in test_w:")
print(test_w.isnull().sum()[test_w.isnull().sum() > 0])

Missing values in train_w:
NumMosquitos              67055
WnvPresent                67055
Id                         8114
trap_percent_with_wnv      6618
trap_percent_with_wnv2    75169
StnPressure                  93
dtype: int64

Missing values in valid_w:
NumMosquitos    27115
WnvPresent      27115
dtype: int64

Missing values in test_w:
NumMosquitos    22123
WnvPresent      22123
Id               2392
dtype: int64


In [ ]:
train_w.NumMosquitos.value_counts()

np.int64(50)